# Multi-Purpose Image Detection and Analysis System

This notebook demonstrates the usage of the image detection and analysis pipeline.


## 1. Setup and Imports


In [ ]:
import sys
from pathlib import Path

# Add parent directory to path
sys.path.insert(0, str(Path().resolve().parent))

import cv2
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

from app.main import ImageAnalysisPipeline
from utils.image_loader import load_image


## 2. Initialize Pipeline


In [ ]:
# Initialize the pipeline with YOLOv8 detector
pipeline = ImageAnalysisPipeline(detector_type="yolo")


## 3. Load and Process Image


In [ ]:
# Load image (replace with your image path)
image_path = "../test_image.jpg"  # Update this path

# Or use a sample image URL
# import urllib.request
# urllib.request.urlretrieve("https://example.com/image.jpg", "sample.jpg")
# image_path = "sample.jpg"

try:
    image = load_image(image_path)
    print(f"Image loaded: {image.shape}")
except FileNotFoundError:
    print(f"Image not found at {image_path}")
    print("Please update the image_path variable with a valid image path")


## 4. Run Analysis


In [ ]:
# Process the image
result = pipeline.process_image(image_path, save_output=True)

# Print summary
print("=" * 50)
print("ANALYSIS SUMMARY")
print("=" * 50)
print(f"Total detections: {result['summary']['total_detections']}")
print(f"Humans: {result['summary']['humans']}")
print(f"Animals: {result['summary']['animals']}")


## 5. Display Results


In [ ]:
# Display original and annotated images
fig, axes = plt.subplots(1, 2, figsize=(15, 7))

# Original image
image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
axes[0].imshow(image_rgb)
axes[0].set_title("Original Image")
axes[0].axis("off")

# Annotated image
if result['annotated_image'] is not None:
    annotated_rgb = cv2.cvtColor(result['annotated_image'], cv2.COLOR_BGR2RGB)
    axes[1].imshow(annotated_rgb)
    axes[1].set_title("Annotated Image")
    axes[1].axis("off")
else:
    axes[1].text(0.5, 0.5, "No detections found", ha="center", va="center")
    axes[1].axis("off")

plt.tight_layout()
plt.show()


## 6. Detailed Results


In [ ]:
# Print detailed results for each detection
for i, detection in enumerate(result['detections'], 1):
    print(f"\n{'=' * 50}")
    print(f"Detection {i}")
    print(f"{'=' * 50}")
    print(f"Type: {detection['class_type']}")
    print(f"Class: {detection.get('class_name', 'N/A')}")
    print(f"Confidence: {detection.get('confidence', 0.0):.2%}")
    print(f"BBox: {detection['bbox']}")
    
    attributes = detection.get('attributes', {})
    if detection['class_type'] == 'human':
        print(f"Age: {attributes.get('age', 'N/A')}")
        print(f"Gender: {attributes.get('gender', 'N/A')}")
        print(f"Emotion: {attributes.get('emotion', 'N/A')}")
    elif detection['class_type'] == 'animal':
        print(f"Species: {attributes.get('species', 'N/A')}")
        print(f"Breed: {attributes.get('breed', 'N/A')}")
        print(f"Maturity: {attributes.get('maturity', 'N/A')}")


## 7. JSON Output


In [ ]:
# Get JSON output
import json

json_output = pipeline.process_image_to_json(image_path)
print(json_output)


## 8. Individual Component Testing


In [ ]:
# Test individual components
from models.yolo_detector import YOLODetector
from models.human_analysis import HumanAnalyzer
from models.animal_analysis import AnimalAnalyzer

# Test detector only
detector = YOLODetector()
detections = detector.detect(image)
print(f"Detected {len(detections)} objects")

# Test human analyzer (if human detected)
human_detections = [d for d in detections if d['class_type'] == 'human']
if human_detections:
    from utils.image_loader import crop_image
    human_analyzer = HumanAnalyzer()
    crop = crop_image(image, human_detections[0]['bbox'])
    human_attrs = human_analyzer.analyze_human(crop)
    print(f"Human attributes: {human_attrs}")

# Test animal analyzer (if animal detected)
animal_detections = [d for d in detections if d['class_type'] == 'animal']
if animal_detections:
    animal_analyzer = AnimalAnalyzer()
    crop = crop_image(image, animal_detections[0]['bbox'])
    species = animal_detections[0].get('class_name', 'unknown')
    animal_attrs = animal_analyzer.analyze_animal(crop, species)
    print(f"Animal attributes: {animal_attrs}")
